In [1]:
!pip install torch torchvision torchaudio
!pip install pycocotools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core

!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 6.7 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Cloning into 'detectron2'...
remote: Enumerating objects: 15837, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 15837 (delta 30), reused 12 (delta 12), pack-reused 15771 (from 2)
Receiving objects: 100% (15837/15837), 6.40 MiB | 9.81 MiB/s, done.
Resolving deltas: 100% (11532/11532), done.
Ignoring dataclasses: markers 'python_version < "3.7"' don't match your en

In [3]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
torch:  2.6 ; cuda:  cu124
detectron2: 0.6


In [4]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Set main directory
ROOT_DIR = "/content/drive/MyDrive/Tree Dataset (Augmentation + Fine Tuned) (Mask R-CNN Canopy) (Vadasan)"

# Define paths for saving outputs
MODEL_DIR = os.path.join(ROOT_DIR, "Model")
FINAL_IMAGES_DIR = os.path.join(ROOT_DIR, "Final images")
COMBINED_DIR = os.path.join(ROOT_DIR, "Combined")

# Create directories if not exist
for dir_path in [MODEL_DIR, FINAL_IMAGES_DIR, COMBINED_DIR]:
    os.makedirs(dir_path, exist_ok=True)

Mounted at /content/drive


In [6]:
import json
import os
from detectron2.structures import BoxMode

# Define dataset paths
DATASET_DIR = "/content/drive/MyDrive/Tree Dataset (Augmentation + Fine Tuned) (Mask R-CNN Canopy) (Vadasan)"
ANNOTATIONS = {
    "train": os.path.join(DATASET_DIR, "train_annotations.coco.json"),
    "valid": os.path.join(DATASET_DIR, "valid_annotations.coco.json"),
    "test": os.path.join(DATASET_DIR, "test_annotations.coco.json"),
}

def load_coco_json(json_file, image_dir):
    with open(json_file) as f:
        coco_data = json.load(f)

    dataset_dicts = []
    for image in coco_data["images"]:
        record = {
            "file_name": os.path.join(image_dir, image["file_name"]),
            "image_id": image["id"],
            "height": image["height"],
            "width": image["width"],
            "annotations": [],
        }

        for ann in coco_data["annotations"]:
            if ann["image_id"] == image["id"]:
                new_category_id = 0  # Ensure category ID is 0 for all annotations

                record["annotations"].append({
                    "bbox": ann["bbox"],
                    "bbox_mode": BoxMode.XYWH_ABS,
                    "category_id": new_category_id,
                    "segmentation": ann["segmentation"],
                })

        dataset_dicts.append(record)

    return dataset_dicts

# Register dataset properly
from detectron2.data import DatasetCatalog, MetadataCatalog

for d in ["train", "valid", "test"]:
    dataset_name = "tree_" + d
    try:
        DatasetCatalog.remove(dataset_name)  # Ensure clean registration
    except KeyError:
        pass  # Ignore if the dataset isn't registered yet

    DatasetCatalog.register(dataset_name, lambda d=d: load_coco_json(
        ANNOTATIONS[d], os.path.join(DATASET_DIR, d)
    ))
    MetadataCatalog.get(dataset_name).set(thing_classes=["Tree"])

# Verify dataset is correctly loaded
for d in ["train", "valid", "test"]:
    dataset_dicts = DatasetCatalog.get("tree_" + d)
    category_ids = set(ann["category_id"] for img in dataset_dicts for ann in img["annotations"])
    print(f"Unique category IDs in {d}: {category_ids}")  # Expected: {0}

Unique category IDs in train: {0}
Unique category IDs in valid: {0}
Unique category IDs in test: {0}


In [7]:
import os
import detectron2.data.transforms as T
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.model_zoo import get_config_file
from detectron2.evaluation import COCOEvaluator
from detectron2.engine import DefaultPredictor
from detectron2.data import DatasetMapper, build_detection_train_loader

# Set output directory
MODEL_DIR = "/content/drive/MyDrive/Tree Dataset (Augmentation + Fine Tuned) (Mask R-CNN Canopy) (Vadasan)"
INFERENCE_DIR = os.path.join(MODEL_DIR, "inference")
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(INFERENCE_DIR, exist_ok=True)

# Load configuration
cfg = get_cfg()
cfg.merge_from_file(get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

# Update configuration
cfg.DATASETS.TRAIN = ("tree_train",)
cfg.DATASETS.TEST = ("tree_valid",)
cfg.DATALOADER.NUM_WORKERS = 4  # Increased for better parallel data loading

cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # 1 class (tree)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

# Solver settings
cfg.SOLVER.IMS_PER_BATCH = 8  # Increased batch size
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.WARMUP_FACTOR = 0.001
cfg.SOLVER.WARMUP_ITERS = 100
cfg.SOLVER.MAX_ITER = 2000  # Increased training iterations for better learning
cfg.SOLVER.STEPS = []  # No LR decay
cfg.SOLVER.CHECKPOINT_PERIOD = 1000  # Save model every 1000 iterations

# Evaluation settings
cfg.TEST.EVAL_PERIOD = 250  # Evaluate every 250 iterations

# Set output directory
cfg.OUTPUT_DIR = MODEL_DIR

# Custom trainer class to include validation and augmentation
class CustomTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return COCOEvaluator(dataset_name, cfg, False, INFERENCE_DIR)

    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=DatasetMapper(cfg, is_train=True, augmentations=[
            T.RandomFlip(horizontal=True, vertical=False),
            T.RandomFlip(horizontal=False, vertical=True),
            T.RandomRotation(angle=[-10, 10]),
            T.RandomBrightness(0.8, 1.2),
            T.RandomContrast(0.8, 1.2),
            T.RandomSaturation(0.8, 1.2),
            T.RandomExtent(scale_range=(0.8, 1.2), shift_range=(0.1, 0.1))  # Fixed
        ]))

# Train model
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# Load trained model for inference
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
predictor = DefaultPredictor(cfg)

print("✅ Training complete. Model loaded for inference.")


[03/19 07:04:05 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
model_final_f10217.pkl: 178MB [00:01, 120MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


[03/19 07:04:07 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[03/19 07:04:55 d2.utils.events]:  eta: 1:06:54  iter: 19  total_loss: 3.064  loss_cls: 0.7079  loss_box_reg: 0.6448  loss_mask: 0.6873  loss_rpn_cls: 0.9047  loss_rpn_loc: 0.1779    time: 2.1130  last_time: 2.3917  data_time: 0.1501  last_data_time: 0.0570   lr: 4.7702e-05  max_mem: 8862M
[03/19 07:05:47 d2.utils.events]:  eta: 1:08:01  iter: 39  total_loss: 2.313  loss_cls: 0.5585  loss_box_reg: 0.7543  loss_mask: 0.6449  loss_rpn_cls: 0.2116  loss_rpn_loc: 0.1606    time: 2.1612  last_time: 2.6602  data_time: 0.0613  last_data_time: 0.1057   lr: 9.7652e-05  max_mem: 8862M
[03/19 07:06:32 d2.utils.events]:  eta: 1:09:26  iter: 59  total_loss: 2.13  loss_cls: 0.4947  loss_box_reg: 0.769  loss_mask: 0.5649  loss_rpn_cls: 0.1632  loss_rpn_loc: 0.1575    time: 2.1956  last_time: 2.1833  data_time: 0.0637  last_data_time: 0.0749   lr: 0.0001476  max_mem: 8862M
[03/19 07:07:18 d2.utils.events]:  eta: 1:09:41  iter: 79  total_loss: 2.007  loss_cls: 0.4517  loss_box_reg: 0.7742  loss_mask: 0

...

[03/19 08:02:09 d2.evaluation.evaluator]: Total inference time: 0:00:00.338197 (0.338197 s / iter per device, on 1 devices)
[03/19 08:02:09 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:00 (0.091487 s / iter per device, on 1 devices)
[03/19 08:02:09 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[03/19 08:02:09 d2.evaluation.coco_evaluation]: Saving results to /content/drive/MyDrive/Tree Dataset (Augmentation + Fine Tuned) (Mask R-CNN Canopy) (Vadasan)/inference/coco_instances_results.json
[03/19 08:02:09 d2.evaluation.coco_evaluation]: Evaluating predictions with official COCO API...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.19s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.359
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] =

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[03/19 08:11:50 d2.evaluation.evaluator]: Total inference time: 0:00:00.224942 (0.224942 s / iter per device, on 1 devices)
[03/19 08:11:50 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:00 (0.090377 s / iter per device, on 1 devices)
[03/19 08:11:50 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[03/19 08:11:50 d2.evaluation.coco_evaluation]: Saving results to /content/drive/MyDrive/Tree Dataset (Augmentation + Fine Tuned) (Mask R-CNN Canopy) (Vadasan)/inference/coco_instances_results.json
[03/19 08:11:50 d2.evaluation.coco_evaluation]: Evaluating predictions with official COCO API...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.09s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.343
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] =

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[03/19 08:21:30 d2.evaluation.evaluator]: Total inference time: 0:00:00.226375 (0.226375 s / iter per device, on 1 devices)
[03/19 08:21:30 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:00 (0.088725 s / iter per device, on 1 devices)
[03/19 08:21:30 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[03/19 08:21:30 d2.evaluation.coco_evaluation]: Saving results to /content/drive/MyDrive/Tree Dataset (Augmentation + Fine Tuned) (Mask R-CNN Canopy) (Vadasan)/inference/coco_instances_results.json
[03/19 08:21:30 d2.evaluation.coco_evaluation]: Evaluating predictions with official COCO API...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.09s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.347
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] =

In [8]:
import torch
import numpy as np
import cv2
import os
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.structures import Instances

# Load trained model
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set threshold for prediction
predictor = DefaultPredictor(cfg)

# Set paths
BINARY_MASK_DIR = "/content/drive/MyDrive/Tree Dataset (Augmentation + Fine Tuned) (Mask R-CNN Canopy) (Vadasan)/Final images"
os.makedirs(BINARY_MASK_DIR, exist_ok=True)

def generate_binary_mask(image_path, save_path):
    image = cv2.imread(image_path)
    outputs = predictor(image)

    instances = outputs["instances"]
    if len(instances) == 0:
        print(f"No detections in {image_path}")
        return

    # Get the segmentation masks
    masks = instances.pred_masks.to("cpu").numpy()

    # Combine all masks into a single binary mask
    binary_mask = np.any(masks, axis=0).astype(np.uint8) * 255

    # Apply morphological closing to refine mask
    kernel = np.ones((5,5), np.uint8)
    binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)

    # Save binary mask
    cv2.imwrite(save_path, binary_mask)

# Process all test images
TEST_IMAGES_DIR = "/content/drive/MyDrive/Tree Dataset (Augmentation + Fine Tuned) (Mask R-CNN Canopy) (Vadasan)/test"
test_images = os.listdir(TEST_IMAGES_DIR)

for img_name in test_images:
    img_path = os.path.join(TEST_IMAGES_DIR, img_name)
    save_path = os.path.join(BINARY_MASK_DIR, f"mask_{img_name}")
    generate_binary_mask(img_path, save_path)

print("✅ Binary masks saved successfully!")

[03/19 08:21:42 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/Tree Dataset (Augmentation + Fine Tuned) (Mask R-CNN Canopy) (Vadasan)/model_final.pth ...
✅ Binary masks saved successfully!


In [5]:
!pip install --no-cache-dir --upgrade rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 209.5 MB/s eta 0:00:00


In [9]:
import rasterio
from rasterio.windows import Window
from PIL import Image
import numpy as np
import cv2
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.structures import Instances

# Set input/output paths
TIF_FILE = "/content/drive/MyDrive/Tree Dataset (Augmentation + Fine Tuned) (Mask R-CNN Canopy) (Vadasan)/vadasan.tif"
OUTPUT_TIF = "/content/drive/MyDrive/Tree Dataset (Augmentation + Fine Tuned) (Mask R-CNN Canopy) (Vadasan)/Combined/final_mask.tif"
OUTPUT_JPG = "/content/drive/MyDrive/Tree Dataset (Augmentation + Fine Tuned) (Mask R-CNN Canopy) (Vadasan)/Combined/final_mask.jpg"

# Read .tif file
with rasterio.open(TIF_FILE) as src:
    width, height = src.width, src.height
    tile_size = 1024  # Split into 1024x1024 chunks

    stitched_mask = np.zeros((height, width), dtype=np.uint8)

    for i in range(0, width, tile_size):
        for j in range(0, height, tile_size):
            # Ensure tile stays within image bounds
            w = min(tile_size, width - i)
            h = min(tile_size, height - j)

            window = Window(i, j, w, h)
            image_tile = src.read(window=window).transpose(1, 2, 0)  # Convert to HxWxC

            # Ensure 3-channel RGB format (convert if needed)
            if image_tile.shape[-1] == 4:  # Convert RGBA → RGB
                image_tile = image_tile[:, :, :3]
            elif image_tile.shape[-1] == 1:  # Grayscale → RGB
                image_tile = np.repeat(image_tile, 3, axis=-1)

            # Convert to BGR (OpenCV format)
            image_tile = cv2.cvtColor(image_tile, cv2.COLOR_RGB2BGR)

            # Convert NumPy image to uint8
            image_tile = image_tile.astype(np.uint8)

            # Run Mask R-CNN
            outputs = predictor(image_tile)  # ⬅️ Keep it as NumPy, do not convert to tensor
            instances = outputs["instances"]

            if len(instances) > 0:
                masks = instances.pred_masks.to("cpu").numpy()
                binary_mask = np.any(masks, axis=0).astype(np.uint8) * 255

                # Store result in the correct region of stitched_mask
                stitched_mask[j:j+h, i:i+w] = binary_mask[:h, :w]

# Save as .tif
with rasterio.open(
    OUTPUT_TIF, 'w',
    driver="GTiff", height=height, width=width,
    count=1, dtype=rasterio.uint8
) as dst:
    dst.write(stitched_mask, 1)

# Save as .jpg
Image.fromarray(stitched_mask).save(OUTPUT_JPG)

print("✅ `.tif` and `.jpg` saved successfully!")

/usr/local/lib/python3.11/dist-packages/rasterio/__init__.py:366: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(


✅ `.tif` and `.jpg` saved successfully!


In [10]:
import os
import cv2
import numpy as np
import rasterio

# Set paths
ROOT_DIR = "/content/drive/MyDrive/Tree Dataset (Augmentation + Fine Tuned) (Mask R-CNN Canopy) (Vadasan)"
TEST_FOLDER = os.path.join(ROOT_DIR, "Test")
FINAL_IMAGES_FOLDER = os.path.join(ROOT_DIR, "Final images")
COMBINED_FOLDER = os.path.join(ROOT_DIR, "Combined")
OVERLAY_FOLDER = os.path.join(ROOT_DIR, "Overlay")

# Ensure overlay folder exists
os.makedirs(OVERLAY_FOLDER, exist_ok=True)

# Function to count trees using contours
def count_trees(binary_mask):
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return len(contours)

# Function to overlay mask on image
def overlay_mask(image_path, mask_path, output_path):
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    if image is None or mask is None:
        print(f"Skipping {image_path}, mask not found.")
        return

    # Convert mask to binary
    binary_mask = (mask > 0).astype(np.uint8) * 255

    # Count trees
    tree_count = count_trees(binary_mask)

    # Overlay mask in red while keeping original colors
    red_mask = np.zeros_like(image)
    red_mask[:, :, 2] = binary_mask  # Apply only to red channel
    overlay = cv2.addWeighted(image, 0.7, red_mask, 0.3, 0)

    # Add tree count text at top-left
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(overlay, f'Trees: {tree_count}', (10, 50), font, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # Save output
    cv2.imwrite(output_path, overlay)

# Process .jpg images (Final images + Test)
for mask_file in os.listdir(FINAL_IMAGES_FOLDER):
    image_file = mask_file.replace("_mask", "")  # Adjust name mapping if necessary
    image_path = os.path.join(TEST_FOLDER, image_file)
    mask_path = os.path.join(FINAL_IMAGES_FOLDER, mask_file)
    output_path = os.path.join(OVERLAY_FOLDER, image_file)

    overlay_mask(image_path, mask_path, output_path)

# Process .tif image (Combined folder)
TIF_FILE = os.path.join(ROOT_DIR, "vadasan.tif")
COMBINED_MASK = os.path.join(COMBINED_FOLDER, "final_mask.tif")
OUTPUT_TIF = os.path.join(OVERLAY_FOLDER, "final_overlay.tif")

if os.path.exists(COMBINED_MASK):
    with rasterio.open(TIF_FILE) as src:
        image = src.read([1, 2, 3]).transpose(1, 2, 0)  # Read RGB channels
        mask = rasterio.open(COMBINED_MASK).read(1)  # Read mask as grayscale

        # Convert image to uint8 format
        image = (image / image.max() * 255).astype(np.uint8)

        # Convert mask to binary
        binary_mask = (mask > 0).astype(np.uint8) * 255

        # Count trees
        tree_count = count_trees(binary_mask)

        # Create a red transparent overlay
        red_mask = np.zeros_like(image)
        red_mask[:, :, 2] = binary_mask  # Apply only to red channel
        overlay = cv2.addWeighted(image, 0.7, red_mask, 0.3, 0)

        # Add tree count text
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(overlay, f'Trees: {tree_count}', (10, 50), font, 2, (0, 0, 255), 5, cv2.LINE_AA)

        # Save as .tif
        with rasterio.open(
            OUTPUT_TIF, "w",
            driver="GTiff", height=overlay.shape[0], width=overlay.shape[1],
            count=3, dtype=rasterio.uint8
        ) as dst:
            for i in range(3):
                dst.write(overlay[:, :, i], i + 1)

    print("✅ TIF Overlay saved!")

print("✅ Overlay process complete!")

Skipping /content/drive/MyDrive/Tree Dataset (Augmentation + Fine Tuned) (Mask R-CNN Canopy) (Vadasan)/Test/mask_chunk_15_jpg.rf.a01fab9918ceba352e30292e52faf62e.jpg, mask not found.
Skipping /content/drive/MyDrive/Tree Dataset (Augmentation + Fine Tuned) (Mask R-CNN Canopy) (Vadasan)/Test/mask_chunk_13_jpg.rf.67527b46d1658dc3746f9ba55317b1ff.jpg, mask not found.
Skipping /content/drive/MyDrive/Tree Dataset (Augmentation + Fine Tuned) (Mask R-CNN Canopy) (Vadasan)/Test/mask_chunk_29_jpg.rf.994b96f79767a3d412f175e187c9d70c.jpg, mask not found.
Skipping /content/drive/MyDrive/Tree Dataset (Augmentation + Fine Tuned) (Mask R-CNN Canopy) (Vadasan)/Test/mask_chunk_3_jpg.rf.afb50919ca59abf604375c41e26e4ad6.jpg, mask not found.
Skipping /content/drive/MyDrive/Tree Dataset (Augmentation + Fine Tuned) (Mask R-CNN Canopy) (Vadasan)/Test/mask_chunk_6_jpg.rf.2dd06e14814d8307516937c7ed06de7a.jpg, mask not found.


/usr/local/lib/python3.11/dist-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


✅ TIF Overlay saved!
✅ Overlay process complete!


In [11]:
import os
import cv2
import numpy as np
import rasterio

# Set paths
ROOT_DIR = "/content/drive/MyDrive/Tree Dataset (Augmentation + Fine Tuned) (Mask R-CNN Canopy) (Vadasan)"
TEST_FOLDER = os.path.join(ROOT_DIR, "Test")
FINAL_IMAGES_FOLDER = os.path.join(ROOT_DIR, "Final images")
COMBINED_FOLDER = os.path.join(ROOT_DIR, "Combined")
OVERLAY_FOLDER = os.path.join(ROOT_DIR, "Overlay")

# Ensure overlay folder exists
os.makedirs(OVERLAY_FOLDER, exist_ok=True)

# Function to count trees using the Canopy Method
def count_trees_canopy(binary_mask):
    """Counts trees using Distance Transform & Watershed segmentation."""
    # Apply Gaussian Blur to smooth small noise
    blurred = cv2.GaussianBlur(binary_mask, (5, 5), 0)

    # Apply threshold to enhance tree canopies
    _, thresh = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY)

    # Distance Transform (Find tree centers)
    dist_transform = cv2.distanceTransform(thresh, cv2.DIST_L2, 5)

    # Normalize the distance transform for visualization
    cv2.normalize(dist_transform, dist_transform, 0, 1.0, cv2.NORM_MINMAX)

    # Threshold to get tree centers
    _, tree_centers = cv2.threshold(dist_transform, 0.3, 1.0, cv2.THRESH_BINARY)
    tree_centers = (tree_centers * 255).astype(np.uint8)

    # Find contours (each representing a tree canopy)
    contours, _ = cv2.findContours(tree_centers, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    return len(contours)  # Number of detected trees

# Function to overlay mask on image
def overlay_mask(image_path, mask_path, output_path):
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    if image is None or mask is None:
        print(f"Skipping {image_path}, mask not found.")
        return

    # Convert mask to binary
    binary_mask = (mask > 0).astype(np.uint8) * 255

    # Count trees using Canopy Method
    tree_count = count_trees_canopy(binary_mask)

    # Overlay mask in red while keeping original colors
    red_mask = np.zeros_like(image)
    red_mask[:, :, 2] = binary_mask  # Apply only to red channel
    overlay = cv2.addWeighted(image, 0.7, red_mask, 0.3, 0)

    # Add tree count text at top-left
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(overlay, f'Trees: {tree_count}', (10, 50), font, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # Save output
    cv2.imwrite(output_path, overlay)

# Process .jpg images (Final images + Test)
for mask_file in os.listdir(FINAL_IMAGES_FOLDER):
    image_file = mask_file.replace("_mask", "")  # Adjust name mapping if necessary
    image_path = os.path.join(TEST_FOLDER, image_file)
    mask_path = os.path.join(FINAL_IMAGES_FOLDER, mask_file)
    output_path = os.path.join(OVERLAY_FOLDER, image_file)

    overlay_mask(image_path, mask_path, output_path)

# Process .tif image (Combined folder)
TIF_FILE = os.path.join(ROOT_DIR, "vadasan.tif")
COMBINED_MASK = os.path.join(COMBINED_FOLDER, "final_mask.tif")
OUTPUT_TIF = os.path.join(OVERLAY_FOLDER, "final_overlay_canopy.tif")

if os.path.exists(COMBINED_MASK):
    with rasterio.open(TIF_FILE) as src:
        image = src.read([1, 2, 3]).transpose(1, 2, 0)  # Read RGB channels
        mask = rasterio.open(COMBINED_MASK).read(1)  # Read mask as grayscale

        # Convert image to uint8 format
        image = (image / image.max() * 255).astype(np.uint8)

        # Convert mask to binary
        binary_mask = (mask > 0).astype(np.uint8) * 255

        # Count trees using Canopy Method
        tree_count = count_trees_canopy(binary_mask)

        # Create a red transparent overlay
        red_mask = np.zeros_like(image)
        red_mask[:, :, 2] = binary_mask  # Apply only to red channel
        overlay = cv2.addWeighted(image, 0.7, red_mask, 0.3, 0)

        # Add tree count text
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(overlay, f'Trees: {tree_count}', (10, 50), font, 2, (0, 0, 255), 5, cv2.LINE_AA)

        # Save as .tif
        with rasterio.open(
            OUTPUT_TIF, "w",
            driver="GTiff", height=overlay.shape[0], width=overlay.shape[1],
            count=3, dtype=rasterio.uint8
        ) as dst:
            for i in range(3):
                dst.write(overlay[:, :, i], i + 1)

    print("✅ TIF Overlay saved!")

print("✅ Overlay process complete!")

Skipping /content/drive/MyDrive/Tree Dataset (Augmentation + Fine Tuned) (Mask R-CNN Canopy) (Vadasan)/Test/mask_chunk_15_jpg.rf.a01fab9918ceba352e30292e52faf62e.jpg, mask not found.
Skipping /content/drive/MyDrive/Tree Dataset (Augmentation + Fine Tuned) (Mask R-CNN Canopy) (Vadasan)/Test/mask_chunk_13_jpg.rf.67527b46d1658dc3746f9ba55317b1ff.jpg, mask not found.
Skipping /content/drive/MyDrive/Tree Dataset (Augmentation + Fine Tuned) (Mask R-CNN Canopy) (Vadasan)/Test/mask_chunk_29_jpg.rf.994b96f79767a3d412f175e187c9d70c.jpg, mask not found.
Skipping /content/drive/MyDrive/Tree Dataset (Augmentation + Fine Tuned) (Mask R-CNN Canopy) (Vadasan)/Test/mask_chunk_3_jpg.rf.afb50919ca59abf604375c41e26e4ad6.jpg, mask not found.
Skipping /content/drive/MyDrive/Tree Dataset (Augmentation + Fine Tuned) (Mask R-CNN Canopy) (Vadasan)/Test/mask_chunk_6_jpg.rf.2dd06e14814d8307516937c7ed06de7a.jpg, mask not found.
✅ TIF Overlay saved!
✅ Overlay process complete!


In [12]:
import os
import cv2
import numpy as np
import rasterio
import geopandas as gpd
from rasterio.transform import from_origin
from shapely.geometry import Polygon

# Set paths
ROOT_DIR = "/content/drive/MyDrive/Tree Dataset (Augmentation + Fine Tuned) (Mask R-CNN Canopy) (Vadasan)"
TIF_PATH = os.path.join(ROOT_DIR, "vadasan.tif")
MASK_PATH = os.path.join(ROOT_DIR, "Combined", "final_mask.tif")
SHAPEFILE_DIR = os.path.join(ROOT_DIR, "Shapefiles")
SHAPEFILE_PATH = os.path.join(SHAPEFILE_DIR, "final_trees.shp")

# Ensure output folder exists
os.makedirs(SHAPEFILE_DIR, exist_ok=True)

# Load mask
with rasterio.open(MASK_PATH) as mask_src:
    mask = mask_src.read(1)  # Read first band (grayscale)
    mask = (mask > 0).astype(np.uint8) * 255  # Convert to binary

# Extract tree contours
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Load original TIF to get georeferencing info
with rasterio.open(TIF_PATH) as tif_src:
    transform = tif_src.transform  # Get spatial transform
    crs = tif_src.crs  # Get coordinate reference system

# Convert contours to geospatial polygons
def convert_to_geo_polygon(contour, transform):
    """Convert image-based contour to a georeferenced polygon."""
    coords = []
    for point in contour:
        img_x, img_y = point[0]  # Image coordinates
        geo_x, geo_y = rasterio.transform.xy(transform, img_y, img_x)  # Convert to geographic coords
        coords.append((geo_x, geo_y))
    return Polygon(coords)

# Create polygons for detected trees
polygons = [convert_to_geo_polygon(cnt, transform) for cnt in contours]

# Create GeoDataFrame
gdf = gpd.GeoDataFrame({"tree_count": list(range(1, len(polygons) + 1))}, geometry=polygons, crs=crs)

# Save as Shapefile
gdf.to_file(SHAPEFILE_PATH, driver="ESRI Shapefile")

print(f"✅ Shapefile saved at: {SHAPEFILE_PATH}")
print(f"🌳 Total Trees Detected: {len(polygons)}")

✅ Shapefile saved at: /content/drive/MyDrive/Tree Dataset (Augmentation + Fine Tuned) (Mask R-CNN Canopy) (Vadasan)/Shapefiles/final_trees.shp
🌳 Total Trees Detected: 369
